<a href="https://colab.research.google.com/github/adaanderson/cokiblue/blob/main/ChessNotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import html
import re
from IPython.display import HTML, display

# TODO:
# validate input to make sure moves are valid
# implement en passant pawn capture

input = '''{sample chess game moves}
1. e4 e5
2. Nf3 d6
3. d4 Bg4
4. d4xe5 Bxf3
5. Qxf3 d6xe5
6. Bc4 Nf6
7. Qb3 Qe7
8. Nc3 c6
9. Bg5 b5
10. Nxb5 c6xb5
11. Bxb5+ Nb8d7 {ambiguous knight piece with Nd7}
12. 0-0-0 Rd8
13. Rxd7 Rxd7
14. Rd1 Qe6
15. Bxd7+ Nxd7
16. Qb8+ Nxb8
17. Rd8#
'''

# Input cleanup
input = re.sub("\{.*\}", r"", input)
input = re.sub("\s+", r" ", input) # replace multiple spaces with one
input = re.sub(r"([0-9]+\.)", r"\n\1", input) # Make a newline before each move number
lines = input.splitlines() # split lines (move pairs) into an array

# Unicode glyphs for pieces
Wking = html.unescape("&#9812")
Bking = html.unescape("&#9818")
Wqueen = html.unescape("&#9813")
Bqueen = html.unescape("&#9819")
Wrook = html.unescape("&#9814")
Brook = html.unescape("&#9820")
Wbishop = html.unescape("&#9815")
Bbishop = html.unescape("&#9821")
Wknight = html.unescape("&#9816")
Bknight = html.unescape("&#9822")
Wpawn = html.unescape("&#9817")
Bpawn = html.unescape("&#9823")

WhitePieces = [Wking, Wqueen, Wbishop, Wrook, Wknight, Wpawn]

# Row and column extents of board
BoardRange = range(0, 8)

# display using html
isHTML = True

# Initialize the chess board
chessboard = [
  [Brook,Bknight,Bbishop,Bqueen,Bking,Bbishop,Bknight,Brook],
  [Bpawn]*8,
  ['_']*8,['_']*8,['_']*8,['_']*8,
  [Wpawn]*8,
  [Wrook,Wknight,Wbishop,Wqueen,Wking,Wbishop,Wknight,Wrook]
  ]

# function to output the chessboard state if using monospaced font
def displayBoard_console(chessboard):
  for row in chessboard:
      print(*row, sep='|')

# function to output the chessboard state if html
def displayBoard(chessboard):
  s = r'<table style="text-align: center;Table-Layout:fixed">'
  s += r'<tr>'
  for col in BoardRange:
    s += r'<td width=20px>'+chr(col+ord('a'))+r'</td>'
  s += r'</tr>'
  for row in BoardRange:
    s += r'<tr>'
    for col in BoardRange:
      piece = chessboard[row][col]
      piece = piece if piece != '_' else ' '
      s += r'<td style="background-color:' + (r'#AAAAAAAA;' if (col+row)&1==1 else r'darkblue;')+r' color:'+(r'white;' if piece in WhitePieces else r'black;')+r'">'+piece+r'</td>'
    s += r'<td>' + str(8-row) + r'</td>'
    s += r'</tr>'
  s += r'</table>'
  return s

# Helper function to search for where pieces came from (since source locations
# are optional).  It returns new srcRow, srcCol by looking for target piece
# starting at dstRow, dstCol and searching (backwards) by deltaRow, deltaCol
# each time along a ray (line).
def searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, deltaRow, deltaCol):
  if srcRow != None and srcCol != None:
    return srcRow, srcCol # already known
  row, col = dstRow + deltaRow, dstCol + deltaCol
  while row in BoardRange and col in BoardRange and chessboard[row][col] == '_':
    row += deltaRow
    col += deltaCol
  if row in BoardRange and col in BoardRange and \
   (srcRow == None or row == srcRow) and \
   (srcCol == None or col == srcCol) and \
   chessboard[row][col] == pieceMoving:
    srcRow, srcCol = row, col
  return srcRow, srcCol

# Helper function to search for where a piece came from among a list of relative
# offsets in array searchPatterns.  It returns new srcRow, srcCol by looking
# for target piece in dstRow, dstCol offset by a pattern delta.
def searchPatternsFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, searchPatterns):
  if srcRow != None and srcCol != None:
    return srcRow, srcCol # already known
  #print("searching patterns:", searchPatterns)
  for pattern in searchPatterns:
    row, col = dstRow + pattern[0], dstCol + pattern[1]
    #print("trying (",row,col,")=", chessboard[row][col])
    if row in BoardRange and col in BoardRange and \
      (srcRow == None or row == srcRow) and \
      (srcCol == None or col == srcCol) and \
      chessboard[row][col] == pieceMoving:
      srcRow, srcCol = row, col
      break
  return srcRow, srcCol

# Given a chessboard, an indication of whose turn it is, and a player's move,
# update the chessboard.
def updateBoard(chessboard, isWhite, move):
  #print(*chessboard,sep='\n')
  #print(['a','b','c','d','e','f','g','h'])
  match move:
    case "0-0-0" | "O-O-O":
      #print("Queen side castle!")
      r = chessboard[7 if isWhite else 0]
      r[4], r[2] = r[2], r[4] # king
      r[3], r[0] = r[0], r[3] # rook
    case "0-0" | "O-O":
      #print("King side castle!")
      r = chessboard[7 if isWhite else 0]
      r[5], r[0] = r[0], r[5] # rook
      r[4], r[6] = r[6], r[4] # king
    case "1/2-1/2":
      print("draw")
    case "0-1":
      print("black wins")
    case "1-0":
      print("white wins")
    case _:
      reMatch = re.match("([KQRBN])?([a-h])?([1-8])?(x)?([a-h])([1-8])(\=[QRBN])?(\+|#)?", move)
      if (reMatch):
        #print(reMatch, reMatch.groups())
        piece = reMatch.group(1)
        piece = piece if piece != None else 'P'
        srcCol = reMatch.group(2)
        srcCol = None if srcCol == None else ord(srcCol[0]) - ord('a')
        srcRow = reMatch.group(3)
        srcRow = None if srcRow == None else 8 - int(srcRow[0])
        isCapture = reMatch.group(4)
        dstCol = reMatch.group(5)
        dstCol = None if dstCol == None else ord(dstCol[0]) - ord('a')
        dstRow = reMatch.group(6)
        dstRow = None if dstRow == None else 8 - int(dstRow[0])
        promo = reMatch.group(7)
        check = reMatch.group(8)
        #print("Piece=",piece)
        match piece:
          case 'P':
            pieceMoving = Wpawn if isWhite else Bpawn
            if (isCapture): # todo: en passant
              searchPatterns = [(1, 1), (1, -1)] if isWhite else [(-1, 1), (-1, -1)]
            elif (not isWhite and dstRow == 3) or (isWhite and dstRow == 4):
              searchPatterns = [(1, 0), (2, 0)] if isWhite else [(-1, 0), (-2, 0)]
            else:
              searchPatterns = [(1, 0)] if isWhite else [(-1, 0)]
            srcRow, srcCol = searchPatternsFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, searchPatterns)

          case 'R':
            pieceMoving = Wrook if isWhite else Brook
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 0, 1) # right
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 0, -1) # left
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 1, 0) # down
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, -1, 0) # up

          case 'B':
            pieceMoving = Wbishop if isWhite else Bbishop
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 1, 1) # up right
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 1, -1) # up left
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, -1, 1) # down right
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, -1, -1) # down left

          case 'N':
            pieceMoving = Wknight if isWhite else Bknight
            searchPatterns = [(2,1), (2,-1), (-2,1), (-2,-1), (1,2), (1,-2), (-1,2), (-1,-2)]
            srcRow, srcCol = searchPatternsFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, searchPatterns)

          case 'K':
            pieceMoving = Wking if isWhite else Bking
            searchPatterns = [(1,1), (1,-1), (-1,1), (-1,-1), (1,0), (-1,0), (0,1), (0,-1)]
            srcRow, srcCol = searchPatternsFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, searchPatterns)

          case 'Q':
            pieceMoving = Wqueen if isWhite else Bqueen
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 1, 1) # up right
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 1, -1) # up left
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, -1, 1) # down right
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, -1, -1) # down left
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 0, 1) # right
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 0, -1) # left
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, 1, 0) # down
            srcRow, srcCol = searchLineFor(pieceMoving, srcRow, srcCol, dstRow, dstCol, -1, 0) # up

        # Update chessboard with piece in new dst position and empty in old src
        chessboard[srcRow][srcCol], chessboard[dstRow][dstCol] = '_', pieceMoving
        if check == '#':
          print("Checkmate;", "white wins:" if isWhite else "black wins:")
        elif check == '+':
          print("black in check:" if isWhite else "white in check:")

      else:
        print("Unrecognized notation:", move)

for line in lines:
  # clean input line
  trimLine = line.strip()
  if trimLine == "": continue
  #input number and pairs
  reNumMoves = re.match("([0-9]+\.)\s+(.*)", trimLine)
  moveNum = reNumMoves.group(1)
  moves = re.split(" ", reNumMoves.group(2))
  # update and display chessboard
  s = ""
  if (isHTML): s += r'<table><tr><td>'
  if len(moves) >= 1:
    whiteMove = moves[0]
    s += moveNum + " White: " + whiteMove
    updateBoard(chessboard, True, whiteMove)
    s += displayBoard(chessboard)
    if len(moves) >= 2: # if white didn't already win...
      blackMove = moves[1]
      if (isHTML): s += r'</td><td>'
      s += moveNum + " Black: " + blackMove
      updateBoard(chessboard, False, blackMove)
      s += displayBoard(chessboard)
  if (isHTML):
    s += r'</td></tr></table>'
    display(HTML(s))


black in check:


black in check:


black in check:


Checkmate; white wins:
